In [135]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly
import re

# Postulantes/Educacion

In [68]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ]
})
valores_educacion.to_csv('../data/adri_data/valor_educacion.csv', sep=',', encoding='utf-8', index=False)
# valores_educacion

In [69]:
# Carga de archivos
post_educacion_1  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion.csv')
#post_educacion.drop('Unnamed: 0', axis=1, inplace=True)

# Carga de archivos
post_educacion_2  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_2.csv')

# Carga de archivos
post_educacion_3  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_3.csv')

post_educacion = pd.concat([post_educacion_1, post_educacion_2, post_educacion_3], axis=0)
# post_educacion.duplicated().sum() # 231551
post_educacion.count()              # 1103193


idpostulante    1103193
nombre          1103193
estado          1103193
dtype: int64

In [70]:
# post_educacion.drop_duplicates(inplace = True)

# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_educacion = pd.merge(post_educacion, valores_educacion, how='inner', \
                         left_on=['nombre','estado'], right_on = ['nombre','estado'])

# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
max_post_educacion.count() # 447909

idpostulante       447909
valor_educacion    447909
dtype: int64

In [71]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner', \
                         left_on=['idpostulante','valor_educacion'], \
                         right_on = ['idpostulante','valor_educacion'])

post_educacion.drop(['nombre', 'estado'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

post_educacion.to_csv('../data/adri_data/postulantes_educacion.csv', sep=',', encoding='utf-8', index=False)
post_educacion.count()

idpostulante       447909
valor_educacion    447909
dtype: int64

# Postulantes/Genero/Edad

In [72]:
post_gen_edad_1 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad.csv')
post_gen_edad_2 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_2.csv')
post_gen_edad_3 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_3.csv')

post_gen_edad = pd.concat([post_gen_edad_1, post_gen_edad_2, post_gen_edad_3], axis=0)
post_gen_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [73]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')
post_gen_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 200887
Data columns (total 3 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null category
dtypes: category(1), object(2)
memory usage: 18.6+ MB


In [74]:
post_gen_edad.drop_duplicates(inplace=True)
post_gen_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [75]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

post_gen_edad.count() 

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [76]:
# Modifico el tipo de dato de la fecha.
post_gen_edad.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

post_gen_edad.to_csv('../data/adri_data/postulantes_genero_edad.csv', sep=',', encoding='utf-8', index=False)

In [77]:
post_gen_edad = pd.read_csv('../data/adri_data/postulantes_genero_edad.csv')
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 
post_gen_edad.count() # 504407 
# post_gen_edad.count() # 505382 

idpostulante    504407
sexo            504407
edad            477883
dtype: int64

In [78]:
post_gen_edad.fillna(-1, inplace=True)
post_gen_edad['edad'] = post_gen_edad['edad'].astype(int)
post_gen_edad

,idpostulante,sexo,edad
0,6MM,2,33
1,Nzz,-1,-1
2,ZX1,-1,-1
3,Nq5,-1,-1
4,ebE,2,65
5,N1x,-1,-1
6,52aw,-1,-1
7,NAjM,1,56
8,eRk9,-1,-1
9,a6OE,-1,-1


# Merge de Postulantes

In [79]:
postulantes = pd.merge(post_educacion, post_gen_edad, how='inner', on='idpostulante')
postulantes.to_csv('../data/adri_data/postulantes.csv', sep=',', encoding='utf-8', index=False)
postulantes.head()

,idpostulante,valor_educacion,sexo,edad
0,8rYD,16,2,43
1,e4VXj,16,2,50
2,a6GvZN,16,2,48
3,5264LM,16,2,41
4,5MlbrN,16,2,40


In [80]:
# postulantes.dropna(axis=0, inplace=True)
# postulantes[(postulantes['sexo'] == -1) | (postulantes['sexo'].isnull())]

In [81]:
# postulantes[postulantes['edad'] == -1]

In [82]:
# postulantes[postulantes.duplicated('idpostulante')]

# Vistas

In [83]:
vistas_1 = pd.read_csv('../data/tp2_data/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_2.csv')
vistas_3 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_3.csv')

vistas = pd.concat([vistas_1, vistas_2, vistas_3], axis=0)
vistas.drop_duplicates(inplace=True)
vistas.rename({'idAviso' : 'idaviso'}, inplace =True, axis=1)
vistas.to_csv('../data/adri_data/vistas.csv', sep=',', encoding='utf-8', index=False)
# vistas.info()

In [84]:
# vistas[vistas['idaviso']==739260]

In [85]:
# vistas = pd.read_csv('../data/adri_data/vistas.csv', parse_dates=['timestamp'])
# vistas.info()

# Postulaciones

In [86]:
postulaciones_1 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones_2.csv')
postulaciones = pd.concat([postulaciones_1, postulaciones_2], axis=0)

postulaciones.drop_duplicates(inplace=True)
postulaciones.to_csv('../data/adri_data/postulaciones.csv', sep=',', encoding='utf-8', index=False)

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


# Avisos Online

In [87]:
avisos_online_1 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online.csv')
avisos_online_2 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online_2.csv')
avisos_online = pd.concat([avisos_online_1, avisos_online_2], axis=0)

avisos_online.drop_duplicates(inplace=True)
avisos_online.to_csv('../data/adri_data/avisos_online.csv', sep=',', encoding='utf-8', index=False)

avisos_online.head()

,idaviso
0,1111718795
1,1110497155
2,1111511125
3,1111930643
4,1112172032


In [88]:
# avisos_online['idaviso'].unique().shape

In [89]:
# top100k = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
#top100k['idaviso'].unique().shape

# Detalle Avisos

In [234]:
avisos_detalles_1 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_2.csv')
avisos_detalles_3 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_3.csv')
avisos_detalles_4 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_4.csv')
avisos_detalles = pd.concat([avisos_detalles_1, avisos_detalles_2, avisos_detalles_3, avisos_detalles_4], axis=0)

In [235]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop_duplicates('idaviso', inplace=True)

In [236]:
avisos_detalles['titulo'] = avisos_detalles['titulo'].apply(lambda x: x.strip())
avisos_detalles.count()

idaviso                 25288
titulo                  25288
descripcion             25288
nombre_zona             25288
ciudad                    160
mapacalle                2069
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [237]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [238]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(14, 10)

In [239]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica','nombre_area'] = 'Bioquimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(0, 10)

In [240]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(9, 10)

In [241]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química','nombre_area'] = 'Quimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(0, 10)

In [242]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(3, 10)

In [243]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social','nombre_area'] = 'Trabajo Social'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(0, 10)

In [244]:
# areas = pd.DataFrame(avisos_detalles['nombre_area'].unique())
# areas.to_csv('../data/adri_data/areas_intermedio.csv', encoding='utf-8', index=False)
# areas

In [245]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [246]:
# avisos_detalles.duplicated().sum()

In [247]:
# avd = avisos_detalles[avisos_detalles.duplicated('idaviso')]['idaviso'].reset_index()
# avd = pd.merge(avd, avisos_detalles)
# avd.to_csv('../data/tp2_data/duplex.csv')

In [248]:
# zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique())
# zona

In [249]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalle_0.csv', sep=',', encoding='utf-8', index=False)

In [250]:
avisos_detalles['titulo'] = avisos_detalles['titulo'].str.lower()
avisos_detalles['titulo'] = avisos_detalles['titulo'].apply(lambda x: x\
    .replace('"', ' ') \
    .replace('*', ' ') \
    .replace('(', ' ') \
    .replace(')', ' ') \
    .replace('!', ' ') \
    .replace('¡', ' ') \
    .replace('¿', ' ') \
    .replace('?', ' ') \
    .replace('-', ' ') \
    .replace('/', ' ') \
    .replace('á', 'a') \
    .replace('é', 'e') \
    .replace('í', 'i') \
    .replace('ó', 'o') \
    .replace('ú', 'u') \
    .replace('ü', 'u') \
    .replace(',', ' ') \
    .replace('.', ' ') \
    .replace('-', ' '))

In [251]:
avisos_detalles['titulo']

0        responsable de marketing producto div  constru...
1                asesor de servicios   postventa chevrolet
2                       analistas de impuestos sr ssr y jr
3                                        software engineer
4                               business developer manager
5                      tecnico quimico mecanico  arroyito 
6        administrativo de creditos y finanzas junior  ...
7                    docente de ingles turno tarde  olivos
8                       m   analista de control de gestion
9                  representantes de cobranzas telefonicas
10                    ingeniero mecanico o electromecanico
11       detectorista y operador para tunelera inteligente
12                           operador mayorista   vendedor
13       se busca vendedoras para distribuidora mayoris...
14                                     atencion al cliente
15                 vendedor de planes de ahorro   camiones
16       telemarketer con experiencia en ventas   turno.

### Detalle Avisos (Titulo)

In [252]:
avisos_detalles_titulo = pd.DataFrame(avisos_detalles['titulo'].unique(), columns=['titulo'])
avisos_detalles_titulo = avisos_detalles_titulo.sort_values('titulo').reset_index(drop=True)
avisos_detalles_titulo.head()

,titulo
0,
1,responsable contable e impositivo
2,diseñadores as graficos quark express
3,etl developer senior
4,functional tester


In [253]:
avisos_detalles_titulo.index.name = 'idtitulo'
avisos_detalles_titulo.to_csv('../data/adri_data/avisos_detalle_titulo.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_titulo = pd.read_csv('../data/adri_data/avisos_detalle_titulo.csv')
avisos_detalles_titulo.count()

titulo    19476
dtype: int64

In [254]:
avisos_detalles_titulo.head()

,titulo
0,responsable contable e impositivo
1,diseñadores as graficos quark express
2,etl developer senior
3,functional tester
4,jovenes profesionales area redaccion it


In [255]:
joined = ' '.join(avisos_detalles_titulo['titulo'])

In [256]:
splitted = joined.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

# splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count()).reset_index()
splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)
splitted = splitted[splitted['palabra'] != '']
splitted['length'] = splitted['palabra'].apply(lambda x: len(x))
# splitted = splitted[splitted['length'] > 4]
# splitted.drop('de', inplace=True)
splitted = splitted[splitted['palabra'] != 'de']
splitted = splitted[splitted['palabra'] != 'para']
splitted = splitted[splitted['palabra'] != 'y']
splitted = splitted[splitted['palabra'] != 'a']
splitted = splitted[splitted['palabra'] != 'en']
splitted = splitted[splitted['palabra'] != 'con']
splitted = splitted[splitted['palabra'] != 'zona']
splitted = splitted[splitted['palabra'] != 'o']

splitted.to_csv('../data/adri_data/palabras_titulo.csv', encoding = 'utf-8', index=False)
splitted = splitted.reset_index(drop=True)
splitted = splitted[splitted['count'] > 500]
splitted

,palabra,count,length
0,analista,2255,8
1,tecnico,1164,7
2,administrativo,931,14
3,vendedor,888,8
4,comercial,817,9
5,ventas,762,6
6,sr,716,2
7,importante,676,10
8,jefe,605,4
9,empresa,583,7


In [257]:
for x in splitted.palabra:
    avisos_detalles['titulo_' + x] = avisos_detalles['titulo'].apply(lambda y: 1 if x in y else 0)

In [258]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,...,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable
0,1111556097,responsable de marketing producto div constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,asesor de servicios postventa chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,analistas de impuestos sr ssr y jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,...,0,0,1,0,0,0,0,1,0,0
3,1112146010,software engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,business developer manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,...,0,0,0,0,0,0,0,0,0,0


In [259]:
# avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_titulo, how='inner', on='titulo')
avisos_detalles.drop('titulo', inplace=True, axis=1)
avisos_detalles.count()

idaviso                  25288
descripcion              25288
nombre_zona              25288
ciudad                     160
mapacalle                 2069
tipo_de_trabajo          25288
nivel_laboral            24953
nombre_area              25288
denominacion_empresa     25281
titulo_analista          25288
titulo_tecnico           25288
titulo_administrativo    25288
titulo_vendedor          25288
titulo_comercial         25288
titulo_ventas            25288
titulo_sr                25288
titulo_importante        25288
titulo_jefe              25288
titulo_empresa           25288
titulo_asistente         25288
titulo_jr                25288
titulo_supervisor        25288
titulo_contable          25288
dtype: int64

In [260]:
avisos_detalles.head()

,idaviso,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_analista,...,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,1,...,0,0,1,0,0,0,0,1,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Nombre Zona)

In [261]:
avisos_detalles_zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique(), columns=['nombre_zona'])
avisos_detalles_zona.index.name = 'idzona'
avisos_detalles_zona.to_csv('../data/adri_data/avisos_detalle_zona.csv', sep=',', encoding='utf-8')
avisos_detalles_zona = pd.read_csv('../data/adri_data/avisos_detalle_zona.csv')
avisos_detalles_zona.head()

,idzona,nombre_zona
0,0,Gran Buenos Aires
1,1,Capital Federal
2,2,GBA Oeste
3,3,Buenos Aires (fuera de GBA)
4,4,La Plata


In [262]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_zona, how='inner', on='nombre_zona')
avisos_detalles.drop('nombre_zona', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_analista,titulo_tecnico,...,titulo_ventas,titulo_sr,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,1,0,...,0,1,0,0,0,0,1,0,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Nombre Ciudad)

In [263]:
avisos_detalles_ciudad = pd.DataFrame(avisos_detalles['ciudad'].unique(), columns=['ciudad'])
avisos_detalles_ciudad.index.name = 'idciudad'
avisos_detalles_ciudad.to_csv('../data/adri_data/avisos_detalle_ciudad.csv', sep=',', encoding='utf-8')
avisos_detalles_ciudad = pd.read_csv('../data/adri_data/avisos_detalle_ciudad.csv')
avisos_detalles_ciudad.head()

,idciudad,ciudad
0,0,NaN
1,1,Buenos Aires
2,2,Argentina
3,3,caba
4,4,paternal


In [264]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_ciudad, how='inner', on='ciudad')
avisos_detalles.drop('ciudad', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_analista,titulo_tecnico,titulo_administrativo,...,titulo_sr,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,1,0,0,...,1,0,0,0,0,1,0,0,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Mapa Calle)

In [265]:
avisos_detalles_mapacalle = pd.DataFrame(avisos_detalles['mapacalle'].unique(), columns=['mapacalle'])
avisos_detalles_mapacalle.index.name = 'idmapacalle'
avisos_detalles_mapacalle.to_csv('../data/adri_data/avisos_detalle_mapacalle.csv', sep=',', encoding='utf-8')
avisos_detalles_mapacalle = pd.read_csv('../data/adri_data/avisos_detalle_mapacalle.csv')
avisos_detalles_mapacalle.head()

,idmapacalle,mapacalle
0,0,NaN
1,1,Guemes 4747
2,2,Comodoro Rivadavia
3,3,Buenos Aires
4,4,Av Hipolito Yrigoyen 13500


In [266]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_mapacalle, how='inner', on='mapacalle')
avisos_detalles.drop('mapacalle', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,...,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Tipo de Trabajo)

In [267]:
avisos_detalles_tipo_de_trabajo = pd.DataFrame(avisos_detalles['tipo_de_trabajo'].unique(), columns=['tipo_de_trabajo'])
avisos_detalles_tipo_de_trabajo.index.name = 'idtipo_de_trabajo'
avisos_detalles_tipo_de_trabajo.to_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv', sep=',', encoding='utf-8')
avisos_detalles_tipo_de_trabajo = pd.read_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv')
avisos_detalles_tipo_de_trabajo.head()

,idtipo_de_trabajo,tipo_de_trabajo
0,0,Full-time
1,1,Part-time
2,2,Temporario
3,3,Por Contrato
4,4,Teletrabajo


In [268]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_tipo_de_trabajo, how='inner', on='tipo_de_trabajo')
avisos_detalles.drop('tipo_de_trabajo', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,nivel_laboral,nombre_area,denominacion_empresa,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,...,titulo_jefe,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle,idtipo_de_trabajo
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Senior / Semi-Senior,Impuestos,Lighthouse Smart,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Nivel Laboral)

In [269]:
avisos_detalles_nivel_laboral = pd.DataFrame(avisos_detalles['nivel_laboral'].unique(), columns=['nivel_laboral'])
avisos_detalles_nivel_laboral.index.name = 'idnivel_laboral'
avisos_detalles_nivel_laboral.to_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv', sep=',', encoding='utf-8')
avisos_detalles_nivel_laboral = pd.read_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv')
avisos_detalles_nivel_laboral.head()

,idnivel_laboral,nivel_laboral
0,0,Senior / Semi-Senior
1,1,Junior
2,2,Jefe / Supervisor / Responsable
3,3,Otro
4,4,Gerencia / Alta Gerencia / Dirección


In [270]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nivel_laboral, how='inner', on='nivel_laboral')
avisos_detalles.drop('nivel_laboral', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,nombre_area,denominacion_empresa,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,...,titulo_empresa,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Ingeniería Eléctrica y Electrónica,Roycan,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Impuestos,Lighthouse Smart,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1112146010,<p> </p><p>Chevron is a multinational company ...,Programación,Chevron Buenos Aires Shared Services Center,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,"<p>Importante empresa de energías renovables, ...",Comercial,RANDSTAD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Detalle Avisos (Nombre Area)

In [271]:
avisos_detalles_nombre_area = pd.DataFrame(avisos_detalles['nombre_area'].unique(), columns=['nombre_area'])
avisos_detalles_nombre_area.sort_values('nombre_area', inplace=True)
avisos_detalles_nombre_area.reset_index(inplace=True)
avisos_detalles_nombre_area.drop('index', axis=1, inplace=True)
avisos_detalles_nombre_area.head()

,nombre_area
0,Abastecimiento
1,Administracion de Seguros
2,Administración
3,Administración de Base de Datos
4,Administración de Personal


In [272]:
avisos_detalles_nombre_area.index.name = 'idnombre_area'
avisos_detalles_nombre_area.to_csv('../data/adri_data/avisos_detalle_nombre_area.csv', sep=',', encoding='utf-8')
avisos_detalles_nombre_area = pd.read_csv('../data/adri_data/avisos_detalle_nombre_area.csv')
avisos_detalles_nombre_area.head()

,idnombre_area,nombre_area
0,0,Abastecimiento
1,1,Administracion de Seguros
2,2,Administración
3,3,Administración de Base de Datos
4,4,Administración de Personal


In [273]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nombre_area, how='inner', on='nombre_area')
avisos_detalles.drop('nombre_area', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,denominacion_empresa,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,...,titulo_asistente,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Sinteplast S.A.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50
1,1112408768,<p>Importante autoservicio mayorista se encuen...,Consultores de Empresas SRL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50
2,1112408781,<p>Importamte cadena de autoservicio mayorista...,Consultores de Empresas SRL,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,50
3,1112344682,<p>En KaizenRH buscamos <strong>Asistente de C...,Kaizen Recursos Humanos,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,50
4,1112279443,<p>Buscamos personas que deseen convertirse en...,Re/Max Net,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50


### Detalle Avisos (Denominacion Empresa)

In [274]:
avisos_detalles_denominacion_empresa = pd.DataFrame(avisos_detalles['denominacion_empresa'].unique(), columns=['denominacion_empresa'])
avisos_detalles_denominacion_empresa.head()

,denominacion_empresa
0,Sinteplast S.A.
1,Consultores de Empresas SRL
2,Kaizen Recursos Humanos
3,Re/Max Net
4,F & C Consultores y Asociados


In [275]:
avisos_detalles_denominacion_empresa.index.name = 'iddenominacion_empresa'
avisos_detalles_denominacion_empresa.to_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv', sep=',', encoding='utf-8')
avisos_detalles_denominacion_empresa = pd.read_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv')
avisos_detalles_denominacion_empresa.head()

,iddenominacion_empresa,denominacion_empresa
0,0,Sinteplast S.A.
1,1,Consultores de Empresas SRL
2,2,Kaizen Recursos Humanos
3,3,Re/Max Net
4,4,F & C Consultores y Asociados


In [276]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_denominacion_empresa, how='inner', on='denominacion_empresa')
avisos_detalles.drop('denominacion_empresa', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,0
1,1112448447,<p>Estamos seleccionando un <strong>vendedor p...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,182,0
2,1112394600,<p>Buscamos un administrativo para incorporars...,0,0,1,0,0,0,0,0,...,0,0,0,1,0,520,0,0,34,0
3,1111266888,<p>Estamos seleccionando un <strong>JEFE DE PR...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,142,0
4,1112426827,<p>Estamos seleccionando un abogado para desem...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,114,0


In [277]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalles.csv', sep=',', encoding='utf-8', index=False)

In [278]:
avisos_detalles.count()

idaviso                   25288
descripcion               25288
titulo_analista           25288
titulo_tecnico            25288
titulo_administrativo     25288
titulo_vendedor           25288
titulo_comercial          25288
titulo_ventas             25288
titulo_sr                 25288
titulo_importante         25288
titulo_jefe               25288
titulo_empresa            25288
titulo_asistente          25288
titulo_jr                 25288
titulo_supervisor         25288
titulo_contable           25288
idzona                    25288
idciudad                  25288
idmapacalle               25288
idtipo_de_trabajo         25288
idnivel_laboral           25288
idnombre_area             25288
iddenominacion_empresa    25288
dtype: int64

In [279]:
avisos_detalles[avisos_detalles['idaviso']==739260]

,idaviso,descripcion,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,titulo_jr,titulo_supervisor,titulo_contable,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
17330,739260,Nuestro cliente es una empresa nac. ubicada en...,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,2,29,854
